In [10]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from patsy import dmatrices
import matplotlib
import matplotlib.pyplot as plt
import scipy
import itertools

In [11]:
def name_model(model):
    my_str = ''
    for i in range(0, len(model.params.keys())):
        if model.params.keys()[i] == 'const':
            my_str += 'const'
        else:
            temp = str(model.params.keys()[i] + 1)
            my_str += 'x' + temp
    return my_str
from itertools import chain
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

In [17]:
df  = pd.read_excel('NFL_Data.xlsx', sheet_name = 'Data')
df_n = df.to_numpy()
X = df_n[:, 1:(df_n.shape[1])]
n = df_n.shape[0]
y = df_n[:, [0]]
#add a column of 1's to the front to account for the constant
X = sm.add_constant(X)
X = pd.DataFrame(X)

28

In [13]:
models_1 = []
for subset in powerset(X.columns):
    if len(subset) >= 1:
        model = sm.OLS(y,X[list(subset)] ).fit() #this is because it looks like perhads a constant is not needed
        models_1.append(model)

In [14]:
len(models_1)
#print(models_1[3].summary())

1023

In [15]:
all_models = pd.DataFrame()
r_squared = []
p = [] #num of regressors
SSRes = []
r2_adj = []
MSRes = []
names = []
AIC = []
BIC = []

for i in range(0,len(models_1)):
    r_squared.append(models_1[i].rsquared)
    SSRes.append(models_1[i].ssr)
    r2_adj.append(models_1[i].rsquared_adj)
    MSRes.append(models_1[i].mse_resid)
    p.append(len(models_1[i].params - 1)) #add -1 if const is included
    names.append(name_model(models_1[i]))
    AIC.append(models_1[i].aic)
    BIC.append(models_1[i].bic)

In [18]:
all_models['r2'] = r_squared
all_models['r2_adj'] = r2_adj
all_models['SSRes'] = SSRes
all_models['MSRes'] = MSRes
all_models['p'] = p

#needed for Cp Statistic
#https://en.wikipedia.org/wiki/Mallows%27s_Cp
s2 = models_1[-1].mse_resid #MSE for the full model
all_models['Cp'] = all_models['SSRes'] /s2 - n + 2 * all_models['p']



all_models['regressors'] = names
all_models['AIC'] = AIC
all_models['BIC'] = BIC


In [106]:
#lower AIC and BIC is better
#can sort models by different nums of included regressors (p) and sort by different statistics
all_models[(all_models['p'] == 4)].sort_values(by=['r2_adj'], ascending=False)

,r2,r2_adj,SSRes,MSRes,p,Cp,regressors,AIC,BIC
349,0.961330,0.954885,65.159116,2.714963,4,-0.547328,x2x7x8x9,111.110134,116.438952
277,0.959049,0.952224,69.002310,2.875096,4,0.600023,x1x2x7x8,112.714751,118.043569
327,0.959031,0.952202,69.033473,2.876395,4,0.609327,x2x3x7x8,112.727394,118.056212
337,0.958768,0.951896,69.475521,2.894813,4,0.741297,x2x4x7x8,112.906117,118.234935
343,0.958689,0.951803,69.609556,2.900398,4,0.781311,x2x5x7x8,112.960084,118.288902
...,...,...,...,...,...,...,...,...,...
203,0.247061,0.152944,246.184136,10.257672,4,53.496076,constx2x3x4,148.329065,153.657883
245,0.212593,0.114167,257.453890,10.727245,4,56.860560,constx4x6x9,149.582369,154.911187
228,0.173659,0.070366,270.184068,11.257670,4,60.661041,constx3x4x9,150.933729,156.262547
225,0.136046,0.028052,282.482027,11.770084,4,64.332487,constx3x4x6,152.180050,157.508868
